# Guided Project: Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the **CIA World Factbook**, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

* population - The population as of 2015.
* population_growth - The annual population growth rate, as a percentage.
* area - The total land and water area.

In this guided project, we'll use SQL in Jupyter Notebook to explore and analyze data from this database. 

In [1]:
# import sys
# !{sys.executable} -m pip install sqlite

### Installing SQL for Jupyter using conda

If you have not installed the SQL for Jupyter

In [2]:
# !conda install -yc conda-forge ipython-sql

In [3]:
%%capture

#loading the database
%load_ext sql 

 #specifying the path to our database
%sql sqlite:///factbook.db

## Overview of the table

In [4]:
# Listing the tables in the factbook database
%sql select * from sqlite_master where type='table'

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We can see that we have only 2 tables in our database
* sqlite_sequence
* facts
<p>We can also the columns in each table and their respective data type</p>

We will be working with the **facts** table. <p>Below is the data dictionary for our table</p>:
* name - The name of the country.
* area - The total land and sea area of the country.
* population - The country's population.
* population_growth- The country's population growth as a percentage.
* birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
* death_rate - The country's death rate, or the number of death a year per 1,000 people.
* area- The country's total area (both land and water).
* area_land - The country's land area in square kilometers.
* area_water - The country's waterarea in square kilometers.

#### Displaying the first 5 rows of *facts* table

In [5]:
%%sql 
select * from facts limit 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

Here we are looking for the following:
* Minimum population
* Maximum population
* Minimum population growth
* Maximum population growth

In [23]:
%%sql 
SELECT MIN(population) AS min_population, 
        MAX(population) AS max_population, 
        MIN(population_growth) AS min_pop_growth, 
        MAX(population_growth) AS max_pop_growth 
    FROM facts;

 * sqlite:///factbook.db
Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


## Exploring Outiers

Using subqueries to know more about the outliers; minimum and maximum population

#### Extracting countries with the minimum population

In [7]:
%%sql
SELECT name 
        FROM facts 
    WHERE population = (SELECT MIN(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
Antarctica


#### Extracting countries with the maximum population

In [8]:
%%sql
SELECT name 
        FROM facts 
    WHERE population = (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
World


In [9]:
%%sql
select * from facts where name = 'Antarctica'

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


The minimum population is 0, this needs further investigation by inspecting the source of the data, [CIA factbook](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html). So we might need to replace appropriately 

Also observably, the world being the maximum population shows that this is total population. Thus, this row needs to be excluded.

## Exploring Average Population and Area

Recalculating the summary statistics by excluding the row for the whole world.

1. Minimum population
2. Maximum population
3. Minimum population growth
4. Maximum population growth

In [10]:
%%sql 
SELECT MIN(population) AS min_population, 
        MAX(population) AS max_population, 
        MIN(population_growth) AS min_pop_growth, 
        MAX(population_growth) AS max_pop_growth 
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


In [11]:
%%sql
SELECT name 
        FROM facts 
    WHERE population = (SELECT MAX(population) 
                                FROM facts 
                            WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


name
China


#### Calculating the average population and area so as to be able to find calculate the densely populated countries

In [12]:
%%sql
SELECT AVG(population) AS avg_population, 
            AVG(area) AS avg_area
        FROM facts
    WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


## Finding Densely Populated Countries
<p>We will be finding the densely populated countries using the code written in the above cell as a subquery here. That is we are looking for countries whose population is above the average population and the area below the average area</p>

###### In order to make the population and area easily readable, we will use thousand separator for the values in those columns

In [13]:
%%sql
select * from facts limit 3

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92


We will also be calulating the [population density](https://www.internetgeography.net/topics/what-is-population-density/) and sort our outcome by the population density

In [14]:
%%sql
SELECT *,  
            printf("%,d", population) AS population, 
            printf("%,d", area) AS area,
            population/area as pop_density
        FROM facts
    WHERE population > (SELECT AVG(population) FROM facts WHERE name <> 'World') 
            AND
        area < (SELECT AVG(area) FROM facts WHERE name <> 'World')
    ORDER BY pop_density DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,population_1,area_1,pop_density
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46,"168,957,745","148,460",1138
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0,"49,115,196","99,720",492
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09,"100,998,376","300,000",336
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0,"126,919,659","377,915",335
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3,"94,348,835","331,210",284
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54,"64,088,222","243,610",263
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24,"80,854,408","357,022",226
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1,"61,855,120","301,340",205
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,"37,101,745","241,038",153
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0,"67,976,405","513,120",132


It can be seen that there are 14 countries that densely populated whihch comprises mostly the Southern America and Asia countries. Also an Africa country is also included in this list. Bangladesh is the most densely populated country in the world followed South Korea.

It can also be observed these densely populated areas are in Asia

### Country with highest population

In [15]:
%%sql
SELECT * 
        FROM facts
    WHERE population = (SELECT MAX(population) 
             FROM facts 
            WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


China is the country with most people with a population 1.38 billion

### Country with the highest growth rate

In [16]:
%%sql
SELECT * 
        FROM facts
    WHERE population_growth = (SELECT MAX(population_growth) 
             FROM facts 
            WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


South Sudan is the country with the highest growth with  population growth rate of 4.02

### Countries with highest ratio of water land

In [17]:
%%sql
select avg(area_water) as avg_water, avg(area_land) as avg_land from facts where name <> 'World'

 * sqlite:///factbook.db
Done.


avg_water,avg_land
19067.59259259259,522702.57723577233


In [18]:
%%sql
select avg(area_water/area_land) from facts where name <> 'World'

 * sqlite:///factbook.db
Done.


avg(area_water/area_land)
3.771784232365145


In [19]:
%%sql
SELECT * 
        FROM facts
    WHERE area_water/area_land > (SELECT AVG(area_water/area_land) FROM facts WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


The countries with the highest water to land ratio are British Indian Ocean Territory

### Countries with more water land

In [20]:
%%sql
SELECT * 
        FROM facts
    WHERE area_water > area_land and name <> 'World'

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


It can be observed the result here is same as the result of the countries with the highest ratio of water to land 

### Which countries will add the most people to their population next year?

Here we are looking for countries with highest population growth rate

In [21]:
%%sql
SELECT *
        FROM facts
    WHERE population_growth > (SELECT AVG(population_growth) 
                                   FROM facts
                               WHERE name <> 'World')
    ORDER BY population_growth DESC

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62


African Countries; **South Sudan, Malawi, Burundi, Niger and Uganda** top the list of countries that will add most people next year

### Countries with higher death rate than birth rate

In [22]:
%%sql
SELECT * 
        FROM facts
    WHERE death_rate > birth_rate
    ORDER BY death_rate DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
